In [ ]:

import hashlib
import os
import warnings
import zipfile
import pandas as pd
import pytz

warnings.filterwarnings('ignore')

# ------------------ CONFIGURATION ------------------
json_data_dir = './download/data'
output_dir = './out/'

marker_dot_size = 0

## -------- some filters to remove entries from the data --------
# e.g. 'APPLE INC', 'Name ABC'
remove_by_name = []
# e.g. 'AAPL', 'MSFT'
remove_by_ticker = []
# e.g. 'Versorger', 'Immobilien', 'Energie', 'Materialien', 'Telekommunikationsdienste'
remove_by_sector = []
# 'Vereinigte Staaten', 'Deutschland', 'Frankreich'
remove_by_standort = []
# ---------------------------------------------------

# -- some checks
if not os.path.exists(json_data_dir):
    raise FileNotFoundError(f"File not found: {json_data_dir}")

# create output directory if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Output directory: {output_dir}")


In [ ]:
import json

# create a hashmap for each day containing the df
df_by_stichtag = {}

# iterate over all files in the data directory
for json_file in os.listdir(json_data_dir):
    if not json_file.endswith('.json'):
        continue

    # Load JSON data
    with open(os.path.join(json_data_dir, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Extract the aaData array
    aaData = data['aaData']

    # Define the column names
    columns = [
        'Emittententicker', 'Name', 'Sektor', 'Anlageklasse', 'Marktwert',
        'Gewichtung (%)', 'Nominalwert', 'Nominale', 'Kurs', 'Standort',
        'Börse', 'Marktwährung'
    ]
    # Create a list of rows
    rows = []
    for item in aaData:
        row = [
            item[0],  # Emittententicker
            item[1],  # Name
            item[2],  # Sektor
            item[3],  # Anlageklasse
            item[4]['raw'],  # Marktwert
            item[5]['raw'],  # Gewichtung (%)
            item[6]['raw'],  # Nominalwert
            item[7]['raw'],  # Nominale
            item[9]['raw'],  # Kurs
            item[10],  # Standort
            item[11],  # Börse
            item[12]  # Marktwährung
        ]
        rows.append(row)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=columns)

    stichtag = json_file.split('.')[0]

    df_by_stichtag[stichtag] = df

print(f"Loaded {len(df_by_stichtag)} dataframes")

# # find the latest entry in the df_by_stichtag by converting the keys to datetime
latest_stichtag = max(df_by_stichtag.keys(), key=lambda x: pd.to_datetime(x, format='%Y%m%d'))
print(f"Latest stichtag: {latest_stichtag}")

#df_by_stichtag

In [ ]:
def get_active_filters_headline():
    filters = []
    if remove_by_name:
        filters.append(f"Name: {', '.join(remove_by_name)}")
    if remove_by_ticker:
        filters.append(f"Ticker: {', '.join(remove_by_ticker)}")
    if remove_by_sector:
        filters.append(f"Sektor: {', '.join(remove_by_sector)}")
    if remove_by_standort:
        filters.append(f"Standort: {', '.join(remove_by_standort)}")

    return 'filtered: [' +  ' | '.join(filters) + ']' if filters else 'no filters applied'

# remove entries by "remove_" lists
for stichtag, df in df_by_stichtag.items():
    # Remove entries by name
    for name in remove_by_name:
        df = df[~df['Name'].str.contains(name, na=False)]

    # Remove entries by ticker
    for ticker in remove_by_ticker:
        df = df[~df['Emittententicker'].str.contains(ticker, na=False)]

    # Remove entries by sector
    for sector in remove_by_sector:
        df = df[~df['Sektor'].str.contains(sector, na=False)]

    # Remove entries by standort
    for standort in remove_by_standort:
        df = df[~df['Standort'].str.contains(standort, na=False)]

    # Update the DataFrame in the dictionary
    df_by_stichtag[stichtag] = df

print(f"Active filters: {get_active_filters_headline()}")

In [ ]:
import random
import matplotlib.pyplot as plt
import pandas as pd

line_styles = ['-', '--', '-.', ':', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted']
line_style_cycle = iter(line_styles)

# Create a dictionary to store the dates and corresponding aggregated "Gewichtung (%)" values for each Standort
standort_entries = {}

# Iterate over the DataFrames to aggregate the "Gewichtung (%)" values for each Standort
for date, df in df_by_stichtag.items():
    for standort in df['Standort'].unique():
        if standort not in standort_entries:
            standort_entries[standort] = {'dates': [], 'weights': []}

        # Sum the "Gewichtung (%)" for the current Standort
        total_weight = df[df['Standort'] == standort]['Gewichtung (%)'].sum()

        standort_entries[standort]['dates'].append(date)
        standort_entries[standort]['weights'].append(total_weight)

# Plot the data
plt.figure(figsize=(12, 8))
for standort, data in standort_entries.items():
    # Create a DataFrame for the current Standort
    standort_df = pd.DataFrame({'Date': data['dates'], 'Gewichtung (%)': data['weights']})

    # Convert the 'Date' column to datetime, invalid parsing will be set as NaT
    standort_df['Date'] = pd.to_datetime(standort_df['Date'], errors='coerce')

    # Drop rows with NaT values in the 'Date' column
    standort_df = standort_df.dropna(subset=['Date'])

    # Sort the DataFrame by date
    standort_df = standort_df.sort_values('Date')

    # get random line style
    random_line_style = random.choice(line_styles)

    # Plot the data for the current Standort
    plt.plot(standort_df['Date'], standort_df['Gewichtung (%)'], marker='o', markersize=marker_dot_size, label=standort, linestyle=random_line_style)

plt.title(f"Total Gewichtung (%) Over Time by Standort | {get_active_filters_headline()}")
plt.xlabel('Date')
plt.ylabel('Total Gewichtung (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
# if filters
# create a unique filename based on the filters
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
plt.savefig(os.path.join(output_dir, f'gewichtung_by_standort_{filters_hash}.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import os
import hashlib
import pandas as pd
import plotly.express as px

# Create a dictionary to store the dates and corresponding aggregated "Gewichtung (%)" values for each Standort
standort_entries = {}

# Iterate over the DataFrames to aggregate the "Gewichtung (%)" values for each Standort
for date, df in df_by_stichtag.items():
    for standort in df['Standort'].unique():
        if standort not in standort_entries:
            standort_entries[standort] = {'dates': [], 'weights': []}

        # Sum the "Gewichtung (%)" for the current Standort
        total_weight = df[df['Standort'] == standort]['Gewichtung (%)'].sum()

        standort_entries[standort]['dates'].append(date)
        standort_entries[standort]['weights'].append(total_weight)

# Create a DataFrame for Plotly
plotly_data = []
for standort, data in standort_entries.items():
    for date, weight in zip(data['dates'], data['weights']):
        plotly_data.append({'Date': date, 'Gewichtung (%)': weight, 'Standort': standort})

plotly_df = pd.DataFrame(plotly_data)

# Convert the 'Date' column to datetime
plotly_df['Date'] = pd.to_datetime(plotly_df['Date'], errors='coerce')

# Drop rows with NaT values in the 'Date' column
plotly_df = plotly_df.dropna(subset=['Date'])

# Sort the DataFrame by date
plotly_df = plotly_df.sort_values('Date')

# Create the Plotly line chart
fig = px.line(plotly_df, x='Date', y='Gewichtung (%)', color='Standort', title=f"Total Gewichtung (%) Over Time by Standort | {get_active_filters_headline()}", category_orders={'Standort': sorted(plotly_df['Standort'].unique())})

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
output_file = os.path.join(output_dir, f'gewichtung_by_standort_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import matplotlib.pyplot as plt

# Identify the top 10 entries by "Gewichtung (%)" in the 20250131 DataFrame
top_10_df = df_by_stichtag[latest_stichtag].nlargest(10, 'Gewichtung (%)')

# Create a dictionary to store the dates and corresponding "Gewichtung (%)" values for each top entry
top_entries = {ticker: {'dates': [], 'weights': []} for ticker in top_10_df['Name']}

# Iterate over the DataFrames to extract the "Gewichtung (%)" values for the top 10 entries
for date, df in df_by_stichtag.items():
    for ticker in top_entries.keys():
        row = df[df['Name'] == ticker]
        if not row.empty:
            top_entries[ticker]['dates'].append(date)
            top_entries[ticker]['weights'].append(row['Gewichtung (%)'].values[0])

# Plot the data
plt.figure(figsize=(12, 8))
for ticker, data in top_entries.items():
    # Create a DataFrame for the current ticker
    ticker_df = pd.DataFrame({'Date': data['dates'], 'Gewichtung (%)': data['weights']})

    # Convert the 'Date' column to datetime, invalid parsing will be set as NaT
    ticker_df['Date'] = pd.to_datetime(ticker_df['Date'], errors='coerce')

    # Drop rows with NaT values in the 'Date' column
    ticker_df = ticker_df.dropna(subset=['Date'])

    # Sort the DataFrame by date
    ticker_df = ticker_df.sort_values('Date')

    # Plot the data for the current ticker
    plt.plot(ticker_df['Date'], ticker_df['Gewichtung (%)'], marker='o', markersize=marker_dot_size, label=ticker)

plt.title(f"Gewichtung (%) Over Time for Top 10 Entries on {latest_stichtag} | {get_active_filters_headline()}")
plt.xlabel('Date')
plt.ylabel('Gewichtung (%)')
plt.legend()
plt.grid(True)
# create a unique filename based on the filters
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
plt.savefig(os.path.join(output_dir, f'gewichtung_top_10_{latest_stichtag}_{filters_hash}.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import plotly.express as px

# Identify the top 10 entries by "Gewichtung (%)" in the latest DataFrame
top_10_df = df_by_stichtag[latest_stichtag].nlargest(10, 'Gewichtung (%)')

# Create a dictionary to store the dates and corresponding "Gewichtung (%)" values for each top entry
top_entries = {ticker: {'dates': [], 'weights': []} for ticker in top_10_df['Name']}

# Iterate over the DataFrames to extract the "Gewichtung (%)" values for the top 10 entries
for date, df in df_by_stichtag.items():
    for ticker in top_entries.keys():
        row = df[df['Name'] == ticker]
        if not row.empty:
            top_entries[ticker]['dates'].append(date)
            top_entries[ticker]['weights'].append(row['Gewichtung (%)'].values[0])

# Create a DataFrame for Plotly
plotly_data = []
for ticker, data in top_entries.items():
    for date, weight in zip(data['dates'], data['weights']):
        plotly_data.append({'Date': date, 'Gewichtung (%)': weight, 'Ticker': ticker})

plotly_df = pd.DataFrame(plotly_data)

# Convert the 'Date' column to datetime
plotly_df['Date'] = pd.to_datetime(plotly_df['Date'], errors='coerce')

# Drop rows with NaT values in the 'Date' column
plotly_df = plotly_df.dropna(subset=['Date'])

# Sort the DataFrame by date
plotly_df = plotly_df.sort_values('Date')

# Create the Plotly line chart
fig = px.line(plotly_df, x='Date', y='Gewichtung (%)', color='Ticker', title=f"Gewichtung (%) Over Time for Top 10 Entries on {latest_stichtag} | {get_active_filters_headline()}", category_orders={'Ticker': sorted(plotly_df['Ticker'].unique())})

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
output_file = os.path.join(output_dir, f'gewichtung_top_10_{latest_stichtag}_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import matplotlib.pyplot as plt

# Extract the corresponding DataFrame
most_recent_df = df_by_stichtag[latest_stichtag]

# Create a scatter plot of the "Gewichtung (%)" values
plt.figure(figsize=(12, 8))
plt.scatter(most_recent_df['Name'], most_recent_df['Gewichtung (%)'], alpha=0.6, s=20)
plt.title(f'Gewichtung (%) for the Most Recent Stichtag: {latest_stichtag} | {get_active_filters_headline()}')
plt.xlabel('')
plt.xticks([])
plt.ylabel('Gewichtung (%)')
plt.grid(True)
plt.tight_layout()
# create a unique filename based on the filters
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
plt.savefig(os.path.join(output_dir, f'gewichtung_scatter_{latest_stichtag}_{filters_hash}.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import plotly.express as px

# Extract the corresponding DataFrame
most_recent_df = df_by_stichtag[latest_stichtag]

# Create a Plotly scatter plot of the "Gewichtung (%)" values
fig = px.scatter(most_recent_df, x='Name', y='Gewichtung (%)', title=f'Gewichtung (%) for the Most Recent Stichtag: {latest_stichtag} | {get_active_filters_headline()}')

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
output_file = os.path.join(output_dir, f'gewichtung_scatter_{latest_stichtag}_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create dictionaries to store the cumulative "Gewichtung (%)" values for each date
cumulative_weights_top_10 = {}
cumulative_weights_top_20 = {}
cumulative_weights_top_50 = {}
cumulative_weights_top_100 = {}

# Iterate over the DataFrames to calculate the cumulative "Gewichtung (%)" for the top 20, top 50, and top 100 entries
for date, df in df_by_stichtag.items():
    # Identify the top 20, top 50, and top 100 entries by "Gewichtung (%)"
    top_10_df = df.nlargest(10, 'Gewichtung (%)')
    top_20_df = df.nlargest(20, 'Gewichtung (%)')
    top_50_df = df.nlargest(50, 'Gewichtung (%)')
    top_100_df = df.nlargest(100, 'Gewichtung (%)')

    # Calculate the cumulative "Gewichtung (%)" for the top 20, top 50, and top 100 entries
    cumulative_weight_top_10 = top_10_df['Gewichtung (%)'].sum()
    cumulative_weight_top_20 = top_20_df['Gewichtung (%)'].sum()
    cumulative_weight_top_50 = top_50_df['Gewichtung (%)'].sum()
    cumulative_weight_top_100 = top_100_df['Gewichtung (%)'].sum()

    # Store the cumulative weights for the current date
    cumulative_weights_top_10[date] = cumulative_weight_top_10
    cumulative_weights_top_20[date] = cumulative_weight_top_20
    cumulative_weights_top_50[date] = cumulative_weight_top_50
    cumulative_weights_top_100[date] = cumulative_weight_top_100

# Convert the cumulative weights dictionaries to DataFrames
cumulative_weights_top_10_df = pd.DataFrame(list(cumulative_weights_top_10.items()),
                                            columns=['Date', 'Cumulative Gewichtung (%)'])
cumulative_weights_top_20_df = pd.DataFrame(list(cumulative_weights_top_20.items()),
                                            columns=['Date', 'Cumulative Gewichtung (%)'])
cumulative_weights_top_50_df = pd.DataFrame(list(cumulative_weights_top_50.items()),
                                            columns=['Date', 'Cumulative Gewichtung (%)'])
cumulative_weights_top_100_df = pd.DataFrame(list(cumulative_weights_top_100.items()),
                                             columns=['Date', 'Cumulative Gewichtung (%)'])

# Convert the 'Date' columns to datetime, invalid parsing will be set as NaT
cumulative_weights_top_10_df['Date'] = pd.to_datetime(cumulative_weights_top_10_df['Date'], errors='coerce')
cumulative_weights_top_20_df['Date'] = pd.to_datetime(cumulative_weights_top_20_df['Date'], errors='coerce')
cumulative_weights_top_50_df['Date'] = pd.to_datetime(cumulative_weights_top_50_df['Date'], errors='coerce')
cumulative_weights_top_100_df['Date'] = pd.to_datetime(cumulative_weights_top_100_df['Date'], errors='coerce')

# Drop rows with NaT values in the 'Date' columns
cumulative_weights_top_10_df = cumulative_weights_top_10_df.dropna(subset=['Date'])
cumulative_weights_top_20_df = cumulative_weights_top_20_df.dropna(subset=['Date'])
cumulative_weights_top_50_df = cumulative_weights_top_50_df.dropna(subset=['Date'])
cumulative_weights_top_100_df = cumulative_weights_top_100_df.dropna(subset=['Date'])

# Sort the DataFrames by date
cumulative_weights_top_10_df = cumulative_weights_top_10_df.sort_values('Date')
cumulative_weights_top_20_df = cumulative_weights_top_20_df.sort_values('Date')
cumulative_weights_top_50_df = cumulative_weights_top_50_df.sort_values('Date')
cumulative_weights_top_100_df = cumulative_weights_top_100_df.sort_values('Date')

# Plot the cumulative "Gewichtung (%)" over time for top 20, top 50, and top 100 entries
plt.figure(figsize=(12, 8))
plt.plot(cumulative_weights_top_10_df['Date'], cumulative_weights_top_10_df['Cumulative Gewichtung (%)'], marker='o',
         markersize=marker_dot_size, label='Top 10')
plt.plot(cumulative_weights_top_20_df['Date'], cumulative_weights_top_20_df['Cumulative Gewichtung (%)'], marker='o',
         markersize=marker_dot_size, label='Top 20')
plt.plot(cumulative_weights_top_50_df['Date'], cumulative_weights_top_50_df['Cumulative Gewichtung (%)'], marker='o',
         markersize=marker_dot_size, label='Top 50')
plt.plot(cumulative_weights_top_100_df['Date'], cumulative_weights_top_100_df['Cumulative Gewichtung (%)'],
         markersize=marker_dot_size, marker='o', label='Top 100')

plt.title(f"Cumulative Gewichtung (%) Over Time for Top 10/20/50/100 Entries | {get_active_filters_headline()}")
plt.xlabel('Date')
plt.ylabel('Cumulative Gewichtung (%)')
plt.legend()
plt.grid(True)
# create a unique filename based on the filters
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
plt.savefig(os.path.join(output_dir, f'gewichtung_cumulative_top_10_20_50_100_{filters_hash}.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import os
import hashlib
import pandas as pd
import plotly.express as px

# Create dictionaries to store the cumulative "Gewichtung (%)" values for each date
cumulative_weights_top_10 = {}
cumulative_weights_top_20 = {}
cumulative_weights_top_50 = {}
cumulative_weights_top_100 = {}

# Iterate over the DataFrames to calculate the cumulative "Gewichtung (%)" for the top 10, top 20, top 50, and top 100 entries
for date, df in df_by_stichtag.items():
    # Identify the top 10, top 20, top 50, and top 100 entries by "Gewichtung (%)"
    top_10_df = df.nlargest(10, 'Gewichtung (%)')
    top_20_df = df.nlargest(20, 'Gewichtung (%)')
    top_50_df = df.nlargest(50, 'Gewichtung (%)')
    top_100_df = df.nlargest(100, 'Gewichtung (%)')

    # Calculate the cumulative "Gewichtung (%)" for the top 10, top 20, top 50, and top 100 entries
    cumulative_weight_top_10 = top_10_df['Gewichtung (%)'].sum()
    cumulative_weight_top_20 = top_20_df['Gewichtung (%)'].sum()
    cumulative_weight_top_50 = top_50_df['Gewichtung (%)'].sum()
    cumulative_weight_top_100 = top_100_df['Gewichtung (%)'].sum()

    # Store the cumulative weights for the current date
    cumulative_weights_top_10[date] = cumulative_weight_top_10
    cumulative_weights_top_20[date] = cumulative_weight_top_20
    cumulative_weights_top_50[date] = cumulative_weight_top_50
    cumulative_weights_top_100[date] = cumulative_weight_top_100

# Convert the cumulative weights dictionaries to DataFrames
cumulative_weights_top_10_df = pd.DataFrame(list(cumulative_weights_top_10.items()), columns=['Date', 'Cumulative Gewichtung (%)'])
cumulative_weights_top_20_df = pd.DataFrame(list(cumulative_weights_top_20.items()), columns=['Date', 'Cumulative Gewichtung (%)'])
cumulative_weights_top_50_df = pd.DataFrame(list(cumulative_weights_top_50.items()), columns=['Date', 'Cumulative Gewichtung (%)'])
cumulative_weights_top_100_df = pd.DataFrame(list(cumulative_weights_top_100.items()), columns=['Date', 'Cumulative Gewichtung (%)'])

# Convert the 'Date' columns to datetime
cumulative_weights_top_10_df['Date'] = pd.to_datetime(cumulative_weights_top_10_df['Date'], errors='coerce')
cumulative_weights_top_20_df['Date'] = pd.to_datetime(cumulative_weights_top_20_df['Date'], errors='coerce')
cumulative_weights_top_50_df['Date'] = pd.to_datetime(cumulative_weights_top_50_df['Date'], errors='coerce')
cumulative_weights_top_100_df['Date'] = pd.to_datetime(cumulative_weights_top_100_df['Date'], errors='coerce')

# Drop rows with NaT values in the 'Date' columns
cumulative_weights_top_10_df = cumulative_weights_top_10_df.dropna(subset=['Date'])
cumulative_weights_top_20_df = cumulative_weights_top_20_df.dropna(subset=['Date'])
cumulative_weights_top_50_df = cumulative_weights_top_50_df.dropna(subset=['Date'])
cumulative_weights_top_100_df = cumulative_weights_top_100_df.dropna(subset=['Date'])

# Sort the DataFrames by date
cumulative_weights_top_10_df = cumulative_weights_top_10_df.sort_values('Date')
cumulative_weights_top_20_df = cumulative_weights_top_20_df.sort_values('Date')
cumulative_weights_top_50_df = cumulative_weights_top_50_df.sort_values('Date')
cumulative_weights_top_100_df = cumulative_weights_top_100_df.sort_values('Date')

# Combine the DataFrames for Plotly
cumulative_weights_top_10_df['Group'] = 'Top 10'
cumulative_weights_top_20_df['Group'] = 'Top 20'
cumulative_weights_top_50_df['Group'] = 'Top 50'
cumulative_weights_top_100_df['Group'] = 'Top 100'
plotly_df = pd.concat([cumulative_weights_top_10_df, cumulative_weights_top_20_df, cumulative_weights_top_50_df, cumulative_weights_top_100_df])

# Create the Plotly line chart
fig = px.line(plotly_df, x='Date', y='Cumulative Gewichtung (%)', color='Group', title=f"Cumulative Gewichtung (%) Over Time for Top 10/20/50/100 Entries | {get_active_filters_headline()}")

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
output_file = os.path.join(output_dir, f'gewichtung_cumulative_top_10_20_50_100_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import os
import hashlib
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Create a dictionary to store the dates and corresponding aggregated "Gewichtung (%)" values for each Anlageklasse
anlageklasse_entries = {}

# Iterate over the DataFrames to aggregate the "Gewichtung (%)" values for each Anlageklasse
for date, df in df_by_stichtag.items():
    for anlageklasse in df['Anlageklasse'].unique():
        if anlageklasse not in anlageklasse_entries:
            anlageklasse_entries[anlageklasse] = {'dates': [], 'weights': []}

        # Sum the "Gewichtung (%)" for the current Anlageklasse
        total_weight = df[df['Anlageklasse'] == anlageklasse]['Gewichtung (%)'].sum()

        anlageklasse_entries[anlageklasse]['dates'].append(date)
        anlageklasse_entries[anlageklasse]['weights'].append(total_weight)

# Plot the data using matplotlib
plt.figure(figsize=(12, 8))
for anlageklasse, data in anlageklasse_entries.items():
    # Create a DataFrame for the current Anlageklasse
    anlageklasse_df = pd.DataFrame({'Date': data['dates'], 'Gewichtung (%)': data['weights']})

    # Convert the 'Date' column to datetime, invalid parsing will be set as NaT
    anlageklasse_df['Date'] = pd.to_datetime(anlageklasse_df['Date'], errors='coerce')

    # Drop rows with NaT values in the 'Date' column
    anlageklasse_df = anlageklasse_df.dropna(subset=['Date'])

    # Sort the DataFrame by date
    anlageklasse_df = anlageklasse_df.sort_values('Date')

    # Plot the data for the current Anlageklasse
    plt.plot(anlageklasse_df['Date'], anlageklasse_df['Gewichtung (%)'], marker='o', markersize=marker_dot_size, label=anlageklasse)

plt.title(f"Total Gewichtung (%) Over Time by Anlageklasse | {get_active_filters_headline()}")
plt.xlabel('Date')
plt.ylabel('Total Gewichtung (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
# create a unique filename based on the filters
filters_hash = hashlib.md5(get_active_filters_headline().encode()).hexdigest()
plt.savefig(os.path.join(output_dir, f'gewichtung_by_anlageklasse_{filters_hash}.png'), dpi=300, bbox_inches='tight')
plt.show()

# Plot the data using plotly
plotly_data = []
for anlageklasse, data in anlageklasse_entries.items():
    for date, weight in zip(data['dates'], data['weights']):
        plotly_data.append({'Date': date, 'Gewichtung (%)': weight, 'Anlageklasse': anlageklasse})

plotly_df = pd.DataFrame(plotly_data)

# Convert the 'Date' column to datetime
plotly_df['Date'] = pd.to_datetime(plotly_df['Date'], errors='coerce')

# Drop rows with NaT values in the 'Date' column
plotly_df = plotly_df.dropna(subset=['Date'])

# Sort the DataFrame by date
plotly_df = plotly_df.sort_values('Date')

# Create the Plotly line chart
fig = px.line(plotly_df, x='Date', y='Gewichtung (%)', color='Anlageklasse', title=f"Total Gewichtung (%) Over Time by Anlageklasse | {get_active_filters_headline()}", category_orders={'Anlageklasse': sorted(plotly_df['Anlageklasse'].unique())})

# Save the Plotly chart as an HTML file
output_file = os.path.join(output_dir, f'gewichtung_by_anlageklasse_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import os
import hashlib
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Create a dictionary to store the dates and corresponding aggregated "Gewichtung (%)" values for each sector
sector_entries = {}

# Iterate over the DataFrames to aggregate the "Gewichtung (%)" values for each sector
for date, df in df_by_stichtag.items():
    for sector in df['Sektor'].unique():
        if sector not in sector_entries:
            sector_entries[sector] = {'dates': [], 'weights': []}

        # Sum the "Gewichtung (%)" for the current sector
        total_weight = df[df['Sektor'] == sector]['Gewichtung (%)'].sum()

        sector_entries[sector]['dates'].append(date)
        sector_entries[sector]['weights'].append(total_weight)

# Plot the data using matplotlib
plt.figure(figsize=(12, 8))
for sector, data in sector_entries.items():
    # Create a DataFrame for the current sector
    sector_df = pd.DataFrame({'Date': data['dates'], 'Gewichtung (%)': data['weights']})

    # Convert the 'Date' column to datetime, invalid parsing will be set as NaT
    sector_df['Date'] = pd.to_datetime(sector_df['Date'], errors='coerce')

    # Drop rows with NaT values in the 'Date' column
    sector_df = sector_df.dropna(subset=['Date'])

    # Sort the DataFrame by date
    sector_df = sector_df.sort_values('Date')

    # Plot the data for the current sector
    plt.plot(sector_df['Date'], sector_df['Gewichtung (%)'], marker='o', markersize=5, label=sector)

plt.title("Total Gewichtung (%) Over Time by Sector")
plt.xlabel('Date')
plt.ylabel('Total Gewichtung (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
# create a unique filename based on the filters
filters_hash = hashlib.md5("sector_analysis".encode()).hexdigest()
plt.savefig(os.path.join(output_dir, f'gewichtung_by_sector_{filters_hash}.png'), dpi=300, bbox_inches='tight')
plt.show()

# Plot the data using plotly
plotly_data = []
for sector, data in sector_entries.items():
    for date, weight in zip(data['dates'], data['weights']):
        plotly_data.append({'Date': date, 'Gewichtung (%)': weight, 'Sector': sector})

plotly_df = pd.DataFrame(plotly_data)

# Convert the 'Date' column to datetime
plotly_df['Date'] = pd.to_datetime(plotly_df['Date'], errors='coerce')

# Drop rows with NaT values in the 'Date' column
plotly_df = plotly_df.dropna(subset=['Date'])

# Sort the DataFrame by date
plotly_df = plotly_df.sort_values('Date')

# Create the Plotly line chart
fig = px.line(plotly_df, x='Date', y='Gewichtung (%)', color='Sector', title="Total Gewichtung (%) Over Time by Sector",
              category_orders={'Sector': sorted(plotly_df['Sector'].unique())})

# Save the Plotly chart as an HTML file
output_file = os.path.join(output_dir, f'gewichtung_by_sector_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Identify the latest date in the dataset
latest_stichtag = max(df_by_stichtag.keys(), key=lambda x: pd.to_datetime(x, format='%Y%m%d'))

# Filter the data for the latest date
latest_df = df_by_stichtag[latest_stichtag]

# Group the data by country and get the top 10 entries by "Gewichtung (%)" for each country
top_10_by_country = latest_df.groupby('Standort').apply(lambda x: x.nlargest(50, 'Gewichtung (%)')).reset_index(drop=True)

# Plot the data
plt.figure(figsize=(14, 10))
for country in top_10_by_country['Standort'].unique():
    country_df = top_10_by_country[top_10_by_country['Standort'] == country]
    plt.barh(country_df['Name'], country_df['Gewichtung (%)'], label=country)

plt.title(f'Top 50 "Name" by "Gewichtung (%)" for Each Country on {latest_stichtag}')
plt.xlabel('Gewichtung (%)')
plt.ylabel('Name')
plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, f'top_50_by_country_{latest_stichtag}.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import os
import hashlib
import pandas as pd
import plotly.express as px

# Identify the latest date in the dataset
latest_stichtag = max(df_by_stichtag.keys(), key=lambda x: pd.to_datetime(x, format='%Y%m%d'))

# Filter the data for the latest date
latest_df = df_by_stichtag[latest_stichtag]

# Group the data by country and get the top 10 entries by "Gewichtung (%)" for each country
top_10_by_country = latest_df.groupby('Standort').apply(lambda x: x.nlargest(50, 'Gewichtung (%)')).reset_index(drop=True)

# Create a DataFrame for Plotly
plotly_data = []
for country in top_10_by_country['Standort'].unique():
    country_df = top_10_by_country[top_10_by_country['Standort'] == country]
    for _, row in country_df.iterrows():
        plotly_data.append({'Name': row['Name'], 'Gewichtung (%)': row['Gewichtung (%)'], 'Country': country})

plotly_df = pd.DataFrame(plotly_data)

# Create the Plotly bar chart
fig = px.bar(plotly_df, x='Gewichtung (%)', y='Name', color='Country', orientation='h', title=f'Top 50 "Name" by "Gewichtung (%)" for Each Country on {latest_stichtag}')

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5("top_50_by_country".encode()).hexdigest()
output_file = os.path.join(output_dir, f'top_50_by_country_{latest_stichtag}_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import os
import hashlib
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Create a dictionary to store the dates and corresponding "Gewichtung (%)" values for each Name
name_entries = {}

# Iterate over the DataFrames to extract the "Gewichtung (%)" values for each Name
for date, df in df_by_stichtag.items():
    for name in df['Name'].unique():
        if name not in name_entries:
            name_entries[name] = {'dates': [], 'weights': []}

        # Get the "Gewichtung (%)" for the current Name
        weight = df[df['Name'] == name]['Gewichtung (%)'].sum()

        name_entries[name]['dates'].append(date)
        name_entries[name]['weights'].append(weight)

# Plot the data using matplotlib
plt.figure(figsize=(12, 8))
for name, data in name_entries.items():
    # Create a DataFrame for the current Name
    name_df = pd.DataFrame({'Date': data['dates'], 'Gewichtung (%)': data['weights']})

    # Convert the 'Date' column to datetime, invalid parsing will be set as NaT
    name_df['Date'] = pd.to_datetime(name_df['Date'], errors='coerce')

    # Drop rows with NaT values in the 'Date' column
    name_df = name_df.dropna(subset=['Date'])

    # Sort the DataFrame by date
    name_df = name_df.sort_values('Date')

    # Plot the data for the current Name
    plt.plot(name_df['Date'], name_df['Gewichtung (%)'], marker='o', markersize=2, label=name)

plt.title("Gewichtung (%) Over Time for Each Name")
plt.xlabel('Date')
plt.ylabel('Gewichtung (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
# create a unique filename based on the filters
filters_hash = hashlib.md5("name_analysis".encode()).hexdigest()
plt.savefig(os.path.join(output_dir, f'gewichtung_by_name_{filters_hash}.png'), dpi=300, bbox_inches='tight')
plt.show()

# Plot the data using plotly
plotly_data = []
for name, data in name_entries.items():
    for date, weight in zip(data['dates'], data['weights']):
        plotly_data.append({'Date': date, 'Gewichtung (%)': weight, 'Name': name})

plotly_df = pd.DataFrame(plotly_data)

# Convert the 'Date' column to datetime
plotly_df['Date'] = pd.to_datetime(plotly_df['Date'], errors='coerce')

# Drop rows with NaT values in the 'Date' column
plotly_df = plotly_df.dropna(subset=['Date'])

# Sort the DataFrame by date
plotly_df = plotly_df.sort_values('Date')

# Create the Plotly line chart
fig = px.line(plotly_df, x='Date', y='Gewichtung (%)', color='Name', title="Gewichtung (%) Over Time for Each Name",
              category_orders={'Name': sorted(plotly_df['Name'].unique())})

# Save the Plotly chart as an HTML file
output_file = os.path.join(output_dir, f'gewichtung_by_name_LARGE_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive Plotly chart saved to {output_file}")

In [ ]:
import pandas as pd
import plotly.express as px
import os

# Aggregate the data by Standort and date
country_weights = []

for date, df in df_by_stichtag.items():
    aggregated = df.groupby('Standort')['Gewichtung (%)'].sum().reset_index()
    aggregated['Date'] = date
    country_weights.append(aggregated)

# Combine all dates into a single DataFrame
country_weights_df = pd.concat(country_weights)

# Convert the 'Date' column to datetime
country_weights_df['Date'] = pd.to_datetime(country_weights_df['Date'], format='%Y%m%d')

# Sort the DataFrame by date
country_weights_df = country_weights_df.sort_values('Date')

# Define a fixed color mapping for each Standort
unique_standorts = country_weights_df['Standort'].unique()
color_mapping = {standort: f'rgba({i * 30 % 255}, {i * 60 % 255}, {i * 90 % 255}, 1)' for i, standort in enumerate(unique_standorts)}

# Create the animated bar chart with fixed colors
fig = px.bar(
    country_weights_df,
    x='Gewichtung (%)',
    y='Standort',
    color='Standort',
    orientation='h',
    animation_frame=country_weights_df['Date'].dt.strftime('%Y-%m-%d'),
    title='All Countries by Weight Over Time',
    labels={'Gewichtung (%)': 'Weight (%)', 'Standort': 'Country'},
    range_x=[0, 100],  # Set the x-axis range to be constant from 0 to 100
    color_discrete_map=color_mapping,  # Apply the fixed color mapping
    category_orders={'Standort': unique_standorts}
)

# Save the animation as an HTML file
filters_hash = hashlib.md5("all_countries_by_gewichtung_time_animation".encode()).hexdigest()
output_file = os.path.join(output_dir, f'all_countries_by_gewichtung_time_animation_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive animation saved to {output_file}")
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import os
import hashlib

# Identify the latest and second latest months
sorted_dates = sorted(df_by_stichtag.keys(), key=lambda x: pd.to_datetime(x, format='%Y%m%d'))
latest_month = sorted_dates[-1]
previous_month = sorted_dates[-2]

# Get the DataFrames for the latest and previous months
latest_df = df_by_stichtag[latest_month]
previous_df = df_by_stichtag[previous_month]

# Add the country to the ticker in brackets
merged_df = pd.merge(
    previous_df[['Name', 'Anlageklasse', 'Gewichtung (%)', 'Standort']],
    latest_df[['Name', 'Anlageklasse', 'Gewichtung (%)', 'Standort']],
    on=['Name', 'Anlageklasse', 'Standort'],
    suffixes=('_previous', '_latest')
)

# Calculate the percentage change
merged_df['Percentage Change (%)'] = ((merged_df['Gewichtung (%)_latest'] - merged_df['Gewichtung (%)_previous']) /
                                      merged_df['Gewichtung (%)_previous']) * 100

# Group by "Anlageklasse" and get the top 20 winners and losers for each group
top_20_winners_per_anlageklasse = merged_df.groupby('Anlageklasse', group_keys=False).apply(
    lambda x: x.nlargest(20, 'Percentage Change (%)')
)
top_20_losers_per_anlageklasse = merged_df.groupby('Anlageklasse', group_keys=False).apply(
    lambda x: x.nsmallest(20, 'Percentage Change (%)')
)

# Combine winners and losers into a single DataFrame
top_20_combined = pd.concat([top_20_winners_per_anlageklasse, top_20_losers_per_anlageklasse])
top_20_combined['Category'] = ['Winner' if x > 0 else 'Loser' for x in top_20_combined['Percentage Change (%)']]

# Add the country to the ticker in brackets
top_20_combined['Name'] = top_20_combined.apply(
    lambda row: f"{row['Name']} ({row['Standort']})", axis=1
)

# Filter out changes lower than 1%
top_20_combined = top_20_combined[abs(top_20_combined['Percentage Change (%)']) >= 1]

# Filter out Anlageklasse groups with no values
top_20_combined = top_20_combined[top_20_combined['Anlageklasse'].notna()]

# Create the Plotly bar chart
fig = px.bar(
    top_20_combined,
    x='Percentage Change (%)',
    y='Name',
    color='Category',
    facet_col='Anlageklasse',
    orientation='h',
    title=f"Top 20 Winners and Losers per Anlageklasse ({previous_month} to {latest_month})",
    labels={'Percentage Change (%)': 'Percentage Change (%)', 'Name': 'Ticker'},
    color_discrete_map={'Winner': 'green', 'Loser': 'red'}
)

# Update the layout to use a logarithmic scale for the x-axis
fig.update_layout(xaxis_type='log')

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5("top_20_winners_losers_per_anlageklasse".encode()).hexdigest()
output_file = os.path.join(output_dir, f'top_20_winners_losers_per_anlageklasse_{latest_month}_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive HTML chart saved to {output_file}")

In [ ]:
import pandas as pd
import plotly.express as px
import os
import hashlib

# Identify the latest and second latest months
sorted_dates = sorted(df_by_stichtag.keys(), key=lambda x: pd.to_datetime(x, format='%Y%m%d'))
latest_month = sorted_dates[-1]
previous_month = sorted_dates[-2]

# Get the DataFrames for the latest and previous months
latest_df = df_by_stichtag[latest_month]
previous_df = df_by_stichtag[previous_month]

# Filter for "Aktien" in the "Anlageklasse" column
latest_df = latest_df[latest_df['Anlageklasse'] == 'Aktien']
previous_df = previous_df[previous_df['Anlageklasse'] == 'Aktien']

# Merge the two DataFrames on 'Name', 'Standort', and 'Anlageklasse' to compare weights
merged_df = pd.merge(
    previous_df[['Name', 'Standort', 'Gewichtung (%)']],
    latest_df[['Name', 'Standort', 'Gewichtung (%)']],
    on=['Name', 'Standort'],
    suffixes=('_previous', '_latest')
)

# Calculate the percentage change
merged_df['Percentage Change (%)'] = ((merged_df['Gewichtung (%)_latest'] - merged_df['Gewichtung (%)_previous']) /
                                      merged_df['Gewichtung (%)_previous']) * 100

# Group by "Standort" and calculate the mean percentage change for each country
grouped_df = merged_df.groupby('Standort', as_index=False)['Percentage Change (%)'].mean()

# Create the Plotly bar chart
fig = px.bar(
    grouped_df,
    x='Standort',
    y='Percentage Change (%)',
    title=f"Percentage Change of 'Aktien' by Country ({previous_month} to {latest_month})",
    labels={'Standort': 'Country', 'Percentage Change (%)': 'Average Percentage Change (%)'},
    color='Standort',
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5("aktien_percentage_change_by_country".encode()).hexdigest()
output_file = os.path.join(output_dir, f'aktien_percentage_change_by_country_{latest_month}_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive HTML chart saved to {output_file}")
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import os
import hashlib

# Identify the latest and second latest months
sorted_dates = sorted(df_by_stichtag.keys(), key=lambda x: pd.to_datetime(x, format='%Y%m%d'))
latest_month = sorted_dates[-1]
previous_month = sorted_dates[-2]

# Get the DataFrames for the latest and previous months
latest_df = df_by_stichtag[latest_month]
previous_df = df_by_stichtag[previous_month]

# Filter for "Geldmarkt" in the "Anlageklasse" column
latest_df = latest_df[latest_df['Anlageklasse'] == 'Geldmarkt']
previous_df = previous_df[previous_df['Anlageklasse'] == 'Geldmarkt']

# Merge the two DataFrames on 'Name', 'Standort', and 'Anlageklasse' to compare weights
merged_df = pd.merge(
    previous_df[['Name', 'Standort', 'Gewichtung (%)']],
    latest_df[['Name', 'Standort', 'Gewichtung (%)']],
    on=['Name', 'Standort'],
    suffixes=('_previous', '_latest')
)

# Calculate the percentage change
merged_df['Percentage Change (%)'] = ((merged_df['Gewichtung (%)_latest'] - merged_df['Gewichtung (%)_previous']) /
                                      merged_df['Gewichtung (%)_previous']) * 100

# Group by "Standort" and calculate the mean percentage change for each country
grouped_df = merged_df.groupby('Standort', as_index=False)['Percentage Change (%)'].mean()

# Create the Plotly bar chart
fig = px.bar(
    grouped_df,
    x='Standort',
    y='Percentage Change (%)',
    title=f"Percentage Change of 'Geldmarkt' by Country ({previous_month} to {latest_month})",
    labels={'Standort': 'Country', 'Percentage Change (%)': 'Average Percentage Change (%)'},
    color='Standort',
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Save the Plotly chart as an HTML file
filters_hash = hashlib.md5("geldmarkt_percentage_change_by_country".encode()).hexdigest()
output_file = os.path.join(output_dir, f'geldmarkt_percentage_change_by_country_{latest_month}_{filters_hash}.html')
fig.write_html(output_file)

print(f"Interactive HTML chart saved to {output_file}")
fig.show()